In [0]:
import logging  # For logging errors or information
import requests  # For making HTTP requests to the weather API
import json  # For handling JSON data
from azure.eventhub import EventHubProducerClient, EventData  # For sending data to Azure Event Hub


# Function to handle the API response
def handle_response(response):  # Handles HTTP response from API
    if response.status_code == 200:  # Checks if response is successful
        return response.json()  # Returns JSON data if successful
    else:
        return f"Error: {response.status_code}, {response.text}"  # Returns error message if not successful


# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):  # Fetches current weather data
    current_weather_url = f"{base_url}/current.json"  # Constructs URL for current weather endpoint
    params = {"key": api_key, "q": location, "aqi": "yes"}  # Sets parameters for API call
    response = requests.get(current_weather_url, params=params)  # Makes GET request to API
    return handle_response(response)  # Handles and returns API response


# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):  # Fetches weather forecast data
    forecast_url = f"{base_url}/forecast.json"  # Constructs URL for forecast endpoint
    params = {
        "key": api_key,  # API key parameter
        "q": location,  # Location parameter
        "days": days,  # Number of days for forecast
    }
    response = requests.get(forecast_url, params=params)  # Makes GET request to API
    return handle_response(response)  # Handles and returns API response


# Function to get Alerts
def get_alerts(base_url, api_key, location):  # Fetches weather alerts
    alerts_url = f"{base_url}/alerts.json"  # Constructs URL for alerts endpoint
    params = {"key": api_key, "q": location, "alerts": "yes"}  # Sets parameters for API call
    response = requests.get(alerts_url, params=params)  # Makes GET request to API
    return handle_response(response)  # Handles and returns API response


# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):  # Flattens and merges API data
    location_data = current_weather.get("location", {})  # Extracts location data
    current = current_weather.get("current", {})  # Extracts current weather data
    condition = current.get("condition", {})  # Extracts weather condition
    air_quality = current.get("air_quality", {})  # Extracts air quality data
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])  # Extracts forecast data
    alert_list = alerts.get("alerts", {}).get("alert", [])  # Extracts alerts list

    flattened_data = {
        "name": location_data.get("name"),  # City name
        "region": location_data.get("region"),  # Region name
        "country": location_data.get("country"),  # Country name
        "lat": location_data.get("lat"),  # Latitude
        "lon": location_data.get("lon"),  # Longitude
        "localtime": location_data.get("localtime"),  # Local time
        "temp_c": current.get("temp_c"),  # Current temperature in Celsius
        "is_day": current.get("is_day"),  # Day or night indicator
        "condition_text": condition.get("text"),  # Weather condition text
        "condition_icon": condition.get("icon"),  # Weather condition icon URL
        "wind_kph": current.get("wind_kph"),  # Wind speed in kph
        "wind_degree": current.get("wind_degree"),  # Wind direction in degrees
        "wind_dir": current.get("wind_dir"),  # Wind direction as text
        "pressure_in": current.get("pressure_in"),  # Pressure in inches
        "precip_in": current.get("precip_in"),  # Precipitation in inches
        "humidity": current.get("humidity"),  # Humidity percentage
        "cloud": current.get("cloud"),  # Cloud cover percentage
        "feelslike_c": current.get("feelslike_c"),  # Feels like temperature in Celsius
        "uv": current.get("uv"),  # UV index
        "air_quality": {  # Air quality details
            "co": air_quality.get("co"),  # Carbon monoxide
            "no2": air_quality.get("no2"),  # Nitrogen dioxide
            "o3": air_quality.get("o3"),  # Ozone
            "so2": air_quality.get("so2"),  # Sulfur dioxide
            "pm2_5": air_quality.get("pm2_5"),  # PM2.5
            "pm10": air_quality.get("pm10"),  # PM10
            "us-epa-index": air_quality.get("us-epa-index"),  # US EPA index
            "gb-defra-index": air_quality.get("gb-defra-index"),  # UK DEFRA index
        },
        "alerts": [
            {
                "headline": alert.get("headline"),  # Alert headline
                "severity": alert.get("severity"),  # Alert severity
                "description": alert.get("desc"),  # Alert description
                "instruction": alert.get("instruction"),  # Alert instructions
            }
            for alert in alert_list  # Loops through all alerts
        ],
        "forecast": [
            {
                "date": day.get("date"),  # Forecast date
                "maxtemp_c": day.get("day", {}).get("maxtemp_c"),  # Max temperature
                "mintemp_c": day.get("day", {}).get("mintemp_c"),  # Min temperature
                "condition": day.get("day", {}).get("condition", {}).get("text"),  # Forecast condition
            }
            for day in forecast  # Loops through all forecast days
        ],
    }
    return flattened_data  # Returns the flattened data


# Main program
def fetch_weather_data():  # Main function to fetch and process weather data
    base_url = "http://api.weatherapi.com/v1/"  # Base URL for weather API
    location = (
        "Queretaro"  # You can replace with any city name based on your preference
    )
    weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")  # Retrieves API key from Databricks secrets

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)  # Gets current weather data
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)  # Gets 3-day forecast data
    alerts = get_alerts(base_url, weatherapikey, location)  # Gets weather alerts

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)  # Flattens and merges all data
    print(json.dumps(merged_data, indent=3))  # Prints merged data as formatted JSON


# Calling the Main Program
fetch_weather_data()  # Executes the main function